Import all package

In [1]:
import numpy as np
import plotly as plt
import yfinance as yfin
import sqlite3 as sql
import pandas as pd
import pandas_datareader as pdd
import threading
import time
import os

Tickers

In [32]:
currencies = ["BTC-USD","ETH-USD","TTE.PA"]

Function to use ticker and get data

In [3]:
def getMetaData(s : str):
    ticker = yfin.Ticker(s)
    ticker.history()
    meta = ticker.history_metadata
    return meta

def getData(s : str):
    ticker = yfin.Ticker(s)
    data = ticker.history("max")
    return data

def getCurrentData(tickers :list):
    return pdd.get_quote_yahoo(tickers)["regularMarketPrice"]

Currency Table with meta data

In [39]:
def createCurrencyTable():
    db = sql.connect("mydatabase.db")
    c = db.cursor()
    dropCurrencyTable = '''DROP TABLE IF EXISTS currency'''

    createCurrencyTable = '''CREATE TABLE currency(
        [currencyId] INTEGER PRIMARY KEY AUTOINCREMENT,
        [currencyName] TEXT,
        [instrumentType] TEXT,
        [regularMarketTime] INT,
        [gtmOffset] INT,
        [timezone] TEXT
        )'''
    c.execute(dropCurrencyTable)
    c.execute(createCurrencyTable)
    db.commit()

def insertMultipleCurrency(currencies : list):
    db = sql.connect("mydatabase.db")
    c = db.cursor()
    request = '''INSERT INTO currency (currencyName,instrumenttype,regularMarketTime,gtmOffset,timezone)
        VALUES  '''
    for curr in currencies:
        metaData = getMetaData(curr)
        request += f'''
        ('{metaData['symbol']}',
        '{metaData['instrumentType']}',
        {metaData['regularMarketTime']},
        {metaData['gmtoffset']},
        '{metaData['timezone']}')'''
        if not curr == currencies[-1]:
            request += ','
        else:
            request += ';'
    
    c.execute(request)
    db.commit()

def insertSoloCurrency(currency : str):
    db = sql.connect("mydatabase.db")
    c = db.cursor()
    metaData = getMetaData(currency)
    request = f'''INSERT INTO currency (currencyName,instrumenttype,regularMarketTime,gtmOffset,timezone)
        VALUES ('{metaData['symbol']}' , '{metaData['instrumentType']}', {metaData['regularMarketTime']} , {metaData['gmtoffset']}, '{metaData['timezone']}')'''
    c.execute(request)
    db.commit()



Currency table

In [37]:
def createAllTable():
    db = sql.connect("mydatabase.db")
    c = db.cursor()
    c.execute('''SELECT currencyName FROM currency''')
    currencies= c.fetchall()
    for curr in currencies:
        tableName = str(curr[0])
        for char in ['-','.','^']:
            tableName = tableName.replace(char,"")
        createNewCurrencyTable = f'''CREATE TABLE IF NOT EXISTS {tableName}(
        [Id] INTEGER PRIMARY KEY AUTOINCREMENT,
        [Close] NUMBER,
        [Volume] NUMBER
        )'''
        c.execute(createNewCurrencyTable)
    db.commit()

def createFillTableRequest(currency : str, tableName : str):
    request = f'''INSERT INTO {tableName} (Close,Volume)
        VALUES  '''
    data = getData(currency)
    dta = pd.DataFrame(data)
    for i in range(dta.shape[0]):
        line = dta.iloc[i]
        request += f'''({round(line['Close'],13)},{line['Volume']})'''
        if not (i == dta.shape[0]-1):
            request += ','
        else:
            request += ';'
    return request

def fillAllTable(currencies : list):
    db = sql.connect("mydatabase.db")
    c = db.cursor()
    for curr in currencies:
        tableName = str(curr)
        for char in ['-','.','^']:
            tableName = tableName.replace(char,"")
        request = createFillTableRequest(curr , tableName)
        c.execute(request)
    db.commit()

Init DB Scipt

In [40]:
createCurrencyTable()
insertMultipleCurrency(currencies)
createAllTable()
fillAllTable(currencies)

Thread get current price

In [30]:
class thread(threading.Thread):
    def __init__(self, threadName , threadTicker):
        threading.Thread.__init__(self)
        self.threadName = threadName
        self.threadTicker = threadTicker
        self.i = 0
        self.killed = False
    
    def run(self):
        print("-------Start Thread--------")
        lastTime = time.time().__int__()
        values = getCurrentData(self.threadTicker)
        print(f'''Value {self.i} : {values.index[0]} , {values[0]}''')
        self.i = 0
        while True and not self.killed:
            currentTime = time.time().__int__() 
            if currentTime == lastTime + 10:
                self.i +=1
                lastTime = currentTime
                values = getCurrentData(self.threadTicker)
                print(f'''Value {self.i} : {values.index[0]} , {values[0]}''')
            if self.i == 10:
                print("-------End Thread--------")
                break
            time.sleep(1)
        return None

    def kill(self):
        self.killed = True
        print("-------Thread Killed---------")

threadApi = thread("getDataThread",["TTE.PA"])
threadApi.start()
threadApi.join()

-------Start Thread--------
Value 0 : TTE.PA , 60.06
Value 1 : TTE.PA , 60.06


KeyboardInterrupt: 

In [31]:
threadApi.kill()

-------Thread Killed---------
